# 정상/비정상 여부를 판단하기 위한 CNN 모델



## 1. 모델 생성

### 1-1. CNN 모델 설계하기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *

import numpy as np

from src import dataset

In [2]:
model = Sequential([
    Conv2D(
        filters=32,
        kernel_size=(5,5),
        padding='same',
        activation='relu',
        input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Conv2D(
        filters=32,
        kernel_size=(3,3),
        padding='same',
        activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.2),

    Dense(16, activation='relu'),
    Dropout(0.2),

    Dense(2, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

### 1-2. 데이터셋 불러오기 & 모델 학습

* 연구 노트: '[2_데이터셋 생성하기.ipynb](./2_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B%20%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0.ipynb./2_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%85%8B%20%EC%83%9D%EC%84%B1%ED%95%98%EA%B8%B0.ipynb)'에서 생성한 데이터를 사용.

In [9]:
(x_train, y_train), (x_test, y_test) = dataset.load()

x_train = np.reshape(x_train / 255.0, tuple([x_train.shape[0]] + list(model.input_shape)[1:]))
x_test  = np.reshape(x_test  / 255.0, tuple([x_test.shape[0]]  + list(model.input_shape)[1:]))

model.fit(x_train, y_train, epochs=24)
model.evaluate(x_test, y_test)

model.save('model/softmax24')

Train on 360 samples
Epoch 1/24
360/360 [==============================] - 1s 2ms/sample - loss: 0.1120 - accuracy: 0.9583
Epoch 2/24
360/360 [==============================] - 1s 1ms/sample - loss: 0.1016 - accuracy: 0.9667
Epoch 3/24
360/360 [==============================] - 0s 1ms/sample - loss: 0.1047 - accuracy: 0.9694
Epoch 4/24
360/360 [==============================] - 0s 1ms/sample - loss: 0.1027 - accuracy: 0.9556
Epoch 5/24
360/360 [==============================] - 0s 1ms/sample - loss: 0.1177 - accuracy: 0.9611
Epoch 6/24
360/360 [==============================] - 0s 1ms/sample - loss: 0.1215 - accuracy: 0.9528
Epoch 7/24
360/360 [==============================] - 0s 1ms/sample - loss: 0.0994 - accuracy: 0.9583
Epoch 8/24
360/360 [==============================] - 0s 1ms/sample - loss: 0.0686 - accuracy: 0.9833
Epoch 9/24
360/360 [==============================] - 0s 1ms/sample - loss: 0.1199 - accuracy: 0.9417
Epoch 10/24
360/360 [==============================] - 0s 1ms

~~ps. 'model/model_32_epochs'은 훈련셋에서 0.9667, 테스트셋에서 0.9222로 오버피팅 발생.  
아래서는 'model/model_16_epochs'를 사용하기로 함.~~

| model | onTrain | onTest | desc |
| --- | --- | --- | --- |
| name(epochs) | accuracy(loss) | accuracy(loss) |  |
| --- | --- | --- | --- |
| softmax8 | 0.9556(0.1455) | 0.9333(0.2052) | Underfitting |
| *softmax16 | 0.9583(0.1052) | 0.9556(0.1011) | *selected |
| softmax24 | 0.9806(0.0496) | 0.9556(0.1601) | Overfitting |

---

## 2. 구현한 모델을 사용하여 예측

In [17]:
from tensorflow.keras.models import load_model

import numpy as np
import cv2

import math
import datetime

from src import dataset

In [20]:
# 위 코드에 이어서...

model = load_model('model/softmax16')

data_list = list(dataset.tmp_load())

mx, my = 0, 0
win_name = 'predict model'
data_info = {
    'basename': '',
    'frame' : {
        'raw': None,
        'mask': None,
        'mask-colored': None
    }
}

# --------------------------------

def onMouse(event, x, y, flags, param):
    """마우스 포인터의 좌표 갱신"""
    global mx, my
    if event == cv2.EVENT_MOUSEMOVE:
        mx, my = x, y

def updateDataset(basename, raw, mask, mask_colored):
    global data_info
    data_info['basename'] = basename
    data_info['frame']['raw'] = raw
    data_info['frame']['mask'] = mask
    data_info['frame']['mask-colored'] = mask_colored

def makePoints(n, cx, cy, min_point, max_point):
    """중심의 좌표(cx,cy)와 한 변의 길이(n)을 이용하여
    정사각형의 두 꼭짓점의 좌표를 계산."""
    x0, y0 = math.floor(cx - n/2), math.floor(cy - n/2)
    x1, y1 = math.floor(cx + n/2), math.floor(cy + n/2)

    # 범위 밖 좌표를 방지하기 위한 보정
    dx, dy = 0, 0
    padding = 4

    minx, miny = min_point
    maxx, maxy = max_point
    
    if (x0 < minx): dx = minx -x0 + padding
    if (x1 > maxx): dx = maxx -x1 - padding

    if (y0 < miny): dy = miny -y0 + padding
    if (y1 > maxy): dy = maxy -y1 - padding

    if (x1-x0) == (y1-y0) != n:
        raise Exception('width height does not match?')

    if dx == dy == 0:
        return x0, y0, x1, y1
    else:
        return makePoints(n, cx+dx, cy+dy, min_point, max_point)

# --------------------------------

cv2.namedWindow(win_name)
cv2.setMouseCallback(win_name, onMouse)

updateDataset(*data_list[0])

# --------------------------------
# 비디오 녹화 준비

height, width = data_info['frame']['raw'].shape[:2]
height_padding = 32
res = (width, height+height_padding)

fourcc = cv2.VideoWriter_fourcc(*'XVID')
now = datetime.datetime.now().strftime("%y-%m-%d_%H-%M-%S")
vw = cv2.VideoWriter(f"data/records/{now}.avi", fourcc, 20.0, res)

# --------------------------------

while True:
    frame_info = data_info['frame']
    # --------------------------------
    # 출력할 프레임 전처리
    h, w = frame_info['raw'].shape[:2]
    x0,y0,x1,y1 = makePoints(28, mx, my, (0,0), (w,h))

    frame_out = np.zeros((h+height_padding, w, 3), dtype=np.uint8)

    frame_cut = cv2.cvtColor(frame_info['raw'][y0:y1, x0:x1], cv2.COLOR_BGR2GRAY)

    frame_bg = frame_info['raw'].copy()
    # frame_bg  = cv2.addWeighted(
    #     frame_info['raw'],          0.75,
    #     frame_info['mask-colored'], 0.75, 0)
    cv2.rectangle(frame_bg, (x0,y0), (x1,y1), (0,0,255), 1)

    frame_out[:h,:] = frame_bg
    frame_out[-30:-2,2:30] = cv2.cvtColor(frame_cut, cv2.COLOR_GRAY2BGR)
    # --------------------------------
    # 모델을 이용한 예측
    x_to_predict = (frame_cut / 255.0).reshape(*model.input_shape[1:])
    y_predicted  = model.predict(np.expand_dims(x_to_predict, axis=0))[0]
    # --------------------------------
    # 출력할 프레임 후처리
    predicted_probability = max(y_predicted)
    predicted_label = np.where(y_predicted == predicted_probability)[0][0]
    predicted_label_title = ['normal', 'broken'][predicted_label]

    cv2.putText(frame_out, f"{predicted_label_title}, Prob: {predicted_probability*100:.4f}%",
                (32, h+12), cv2.FONT_HERSHEY_PLAIN, 1.0, (0,0,255), lineType=cv2.LINE_AA)

    cv2.putText(frame_out, str(y_predicted),
                (32, h+24), cv2.FONT_HERSHEY_PLAIN, 0.8, (0,0,255), lineType=cv2.LINE_AA)
    # --------------------------------
    cv2.imshow(win_name, frame_out)
    vw.write(frame_out)

    key = chr(cv2.waitKey(10) & 0xFF)
    if key == 'q':
        break

vw.release()
cv2.imshow(win_name, frame_out)
cv2.destroyWindow(win_name)

print(f"video file saved at [data/records/{now}.avi].")

video file saved at [data/records/07_07-18-30.avi].
